## Linear programming using pulp lib - knapsack problem

#### This script exemplifies a problem of filling up a knapsack with travel items - each with an importance value number and mass, by modeling an objective function (maximum importance value items packed), based on decision variables (binary format: 1 - included or 0 - excluded) and some restrictions regarding the total mass of the final knapsack. The objective is to find the most important items that should be included (and the least important ones that should not), so that the maximum importance value sum is reached, for the final set of packaged items. Part of restrictions concerns to the "weight" (mass in grams) of the total set of items included. It should be included the most important items but limited to the maximum total weight. After modeling, the solver is used to solve the problem and the results are returned with the final sum of importance values of the items included (objective function's value), as well as the binary flag of inclusion (or not) of each individual item - 0 or 1 (decision variables' values). Here, no evolutionary computation is done (including genetic algorithms), only traditional deterministic linear programming, returning always the same optimal result. For a bigger number of variables, this problem would turn difficult to be computed, if not impossible. Here, only nine decision variables are used.

In [7]:
pip install pulp --quiet # do NOT use Conda for installing this lib module. Only works with pip install

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pulp

In [9]:
# creates the linear programming (Lp) problem, setting the problem name and the sense of the problem objective, in such case, 
# a maximization problem
problem = pulp.LpProblem("knapsack", pulp.LpMaximize)

# creates the decision variables for the Lp problem, setting a name for each of those variables and a binary bounds restriction - 
# 0 or 1)
x1 = pulp.LpVariable("cereal_bar", lowBound = 0, upBound = 1, cat = "Binary")
x2 = pulp.LpVariable("coat", lowBound = 0, upBound = 1, cat = "Binary")
x3 = pulp.LpVariable("tennis", lowBound = 0, upBound = 1, cat = "Binary")
x4 = pulp.LpVariable("cell_phone", lowBound = 0, upBound = 1, cat = "Binary")
x5 = pulp.LpVariable("water_bottle", lowBound = 0, upBound = 1, cat = "Binary")
x6 = pulp.LpVariable("sunscreen", lowBound = 0, upBound = 1, cat = "Binary")
x7 = pulp.LpVariable("lip_balm", lowBound = 0, upBound = 1, cat = "Binary")
x8 = pulp.LpVariable("oxygen_bottles", lowBound = 0, upBound = 1, cat = "Binary")
x9 = pulp.LpVariable("camera", lowBound = 0, upBound = 1, cat = "Binary")

# defines the objective function, which is based on the decision variables (0 or 1, from x1 until x9), having as coefficients the 
# importance value of each individual item. The final value for the function represents the sum of all importance values for all 
# included items, according to the problem solution. Non-included items will zero its respective addend (and its importance value 
# as well). Only the importance values of included items will be summed up.
problem += 6*x1 + 7*x2 + 3*x3 + 2*x4 + 9*x5+ 5*x6 + 2*x7 + 10*x8 + 6*x9, "Total_value"

# other than the binary restriction already set above, the other restriction would be: the sum of the mass of all items included 
# inside the knapsack must not surpass 5000 grams. Coefficients here, instead of the importance value above, represent the mass
# (in grams) of each item (which may be included or not). It totalizes only the total mass of items that are ultimately included, 
# according to the problem solution.
problem += 200*x1 + 400*x2 + 400*x3 + 100*x4 + 1000*x5+ 200*x6 + 30*x7 + 3000*x8 + 500*x9 <=5000, "Total_mass"

# initializing and running the solver with the data set at the Lp model. The msg arg here is simply to ommit the verbose 
# calculation. The results are stored at the problem object for later view.
solver = pulp.PULP_CBC_CMD(msg=False)
problem.solve(solver)

1

In [10]:
# Showing the results stored at the problem object, including the status of the problem resolution, the objective function (total 
# maximum importance value of included items) and the inclusion or not (1 or 0) of each product for that final optimal knapsack 
# filling set (decision variables)
print("Results:")
print(f"Status: {pulp.LpStatus[problem.status]}")
print(f"Maximum importance value sum: {int(pulp.value(problem.objective))}")
print(f"Cereal bar: {'✅' if x1.varValue == 1 else '❌'}")
print(f"Coat: {'✅' if x2.varValue == 1 else '❌'}")
print(f"Tennis: {'✅' if x3.varValue == 1 else '❌'}")
print(f"Cell phone: {'✅' if x4.varValue == 1 else '❌'}")
print(f"Water bottle: {'✅' if x5.varValue == 1 else '❌'}")
print(f"Sunscreen: {'✅' if x6.varValue == 1 else '❌'}")
print(f"Lip balm: {'✅' if x7.varValue == 1 else '❌'}")
print(f"Oxygen bottles: {'✅' if x8.varValue == 1 else '❌'}")
print(f"Camera: {'✅' if x9.varValue == 1 else '❌'}")

Results:
Status: Optimal
Maximum importance value sum: 41
Cereal bar: ✅
Coat: ✅
Tennis: ✅
Cell phone: ✅
Water bottle: ❌
Sunscreen: ✅
Lip balm: ✅
Oxygen bottles: ✅
Camera: ✅


#### As a conclusion, and considering the maximization of value of items included at the knapsack, we could include all except the water bottle, and the restriction <=5000g would have been obeyed. The maximum value for the whole knapsack would be 41. If you change the coefficients at the restriction (mass) or at the objective function (importance value), for one or more of the items, the result will change as well, and the included items set will differ. We have another solution, which would be to include all but the tennis and the camera items. The objective function would also be 41 in value, and the <=5000g restriction would still be obeyed. The only prioritization here is regarding the objective function total value, there's no prioritization regarding the mass (e.g. the lowest possible total mass), as long as the total mass keeps <= 5000g.